In [36]:
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table

In [37]:
# dr2_s = Table.read('/media/walml/beta/galaxy_zoo/decals/dr1_dr2/subjects/decals_dr1_and_dr2_with_subj_id.csv').to_pandas()
# dr2_s = dr2_s[['dr8objid', 'sdss_id']]
dr2_s = pd.read_csv('../../dr1_dr2_with_extra_checks.csv')  # subject catalog, could have been classified
dr2_c = pd.read_csv('../../dr2_aggregated_votes_good_subjects.csv')  # aggregated classifications with bad images removed, actually were classified

/home/walml/anaconda3/envs/zoobot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/walml/anaconda3/envs/zoobot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (93) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [38]:
dr2_s['jpeg_exists'] = dr2_s['dr1_jpeg_exists'] | dr2_s['dr2_jpeg_exists']

In [39]:
dr2_quality = dr2_s.groupby('iauname').agg({'jpeg_exists': 'any', 'good_subject_image': any})

In [40]:
len(dr2_quality)

101299

In [41]:
dr2_quality['jpeg_exists'].value_counts()  # I have to assume these 7.2k had bad images by the 20% missing over all bands check

True     94071
False     7228
Name: jpeg_exists, dtype: int64

In [42]:
dr2_quality['good_subject_image'].value_counts()  # and then the extra 1k are from me increasing the check to 20% in *any* band

True     93007
False     8292
Name: good_subject_image, dtype: int64

In [43]:
1 - dr2_quality['good_subject_image'].mean()

0.08185668170465654

In [44]:
len(dr2_c), len(dr2_quality.query('good_subject_image'))  # 47 remain with good images but unclassified, may backfill these next data release but it's too late now. Consider these as not attempted to be included in DR2.

(92960, 93007)

For the final DR2 number, let's use: galaxies which had good images and were classified (92960) / galaxies which tried to be included in DR2 (101252, i.e. 101299 - 47)

In [45]:
92960/101252

0.9181053213763679

In [48]:
to_upload = Table.read('/media/walml/beta/galaxy_zoo/decals/catalogs/dr5_nsa1_0_0_to_upload.fits').to_pandas()  # same table, now with png_ready/fits_ready/fits_filled cols added
to_upload['iauname'] = to_upload['iauname'].str.decode('utf8')
len(to_upload)

343128

In [49]:
 to_upload['fits_filled'].mean(),

(0.9016984915250286,)

In [50]:
to_upload['fits_ready'].mean(), to_upload['png_ready'].mean()  # not working properly

(1.0, 1.0)

In [60]:
to_upload['considered_in_dr2'] = to_upload['iauname'].isin(dr2_s['iauname'])  # using s because these galaxies were never considered for upload
# to_upload['in_dr2'] = to_upload['iauname'].isin(dr2_c['iauname'])
to_upload['considered_in_dr2'].sum()  # 5k fewer, as some were in subjects but never uploaded (bad images, at least mostly)

95382

In [62]:
# dropped in DR5 because of bad fits
new_for_dr5 = to_upload[~to_upload['considered_in_dr2']]
num_new = len(new_for_dr5)
num_good_new = len(new_for_dr5.query('fits_filled'))

num_new, num_good_new, num_good_new/num_new

(247746, 216106, 0.8722885535992508)

In [56]:
maybe - cant

311488

In [57]:
# only true if every galaxy in to_upload is eventually classified, be careful to check this

How many galaxies should we have classified?

In [58]:
to_upload['fits_filled'].sum()

309398